In [ ]:
%load_ext autoreload
%autoreload 2

import uproot

In [ ]:
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import seaborn as sns
import pandas as pd

import os

from analysis_variables import *  
import babar_analysis_tools as bat

In [ ]:
# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

#Josie Laptop
topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"

# At home (Bellis)
#topdir = '/home/bellis/babar_data/bnv_plambda'

# Background
filename= f"{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet"
#filename1 = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

start = time.time()

alldata = ak.from_parquet(filename)

data= alldata[alldata["spmode"]!="-999"]
signal= alldata[alldata["spmode"]=="-999"]

print(f"Took {time.time() - start} s")

print(type(data))
print(data)

In [ ]:
data["BpostFitDeltaE"]

In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

In [ ]:
dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes



In [ ]:
bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)

In [ ]:
all_hists= bat.create_empty_histograms(hist_defs)


In [ ]:
x = list(all_hists.keys())
print(x)

In [ ]:

'''
subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS', \
         'Lambda0_unc_Mass', 'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']
'''
#subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS']
subset = ['Lambda0_unc_Mass']


bkg_modes =['998', '1005', '1235', '1237', '3981']
sig_modes = ['-999']

cut_idx = -1
cut_name = str(cut_idx)
cut = dcuts[cut_idx]['event']

df = bat.fill_histograms_v3(data[cut], subset=subset, empty_hists=all_hists, spmodes=bkg_modes + sig_modes, weights=weights, cutname=cut_name)

#df = bat.fill_histograms_v3(data, subset=['BpostFitDeltaE'], empty_hists=all_hists, spmodes=bkg_modes + sig_modes, weights=weights)


In [ ]:
all_vars = list(all_hists.keys())

#print(all_vars)
#print(len(all_vars))
#print(len(subset))

bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=bkg_modes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=False, only_stacked=True, save= True, fixed_grid=(4,5));

In [ ]:
cut_idx = 1
cut_name = str(cut_idx)
cut = dcuts[cut_idx]['event']

In [ ]:
#### BACKGROUNDS LAMBDA MASS 

df = bat.fill_histograms_v3(data[cut], subset=subset, empty_hists=all_hists, spmodes=bkg_modes + sig_modes, weights=weights, cutname=cut_name)
bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=bkg_modes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=False, only_stacked=True, save= True, fixed_grid=(4,5));


In [ ]:
df = bat.fill_histograms_v3(signal[cut], subset=subset, empty_hists=all_hists, spmodes=sig_modes, weights=weights, cutname=cut_name)
bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=sig_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=False, only_stacked=True, save= True, fixed_grid=(4,5));
